# PANDAS ALTERNATIVES IN PYTHON

- System: 

    Mackbook Pro 15 Inch, 2019 
        
        Processor: 2.3 GHz 8-Core Intel Core i9
        
        Memory: 16 GB 2400 MHz DDR4
        
        macOS:  Sonoma 14.5 Beta (23F5049f)

    Python 3.12.2

____

- Data source: 

    Western Pennsylvania Regional Data Center

    Cumulative Crash Data -->  https://data.wprdc.org/dataset/allegheny-county-crash-data/resource/2c13021f-74a9-4289-a1e5-fe0472c89881

    Given the size of the original dataset is larger than normal limit in github, this command can be run from the notebook to download the data and generate files of different sizes: 

                exec(open("get_data.py").read()) 
____

### __POLARS__:

- Documentation 

    https://docs.pola.rs/py-polars/html/reference/dataframe/api/polars.DataFrame.clone.html

- Installation

    !pip install polars

______ 

### __DATATABLE__:

- Documentation 

    https://datatable.readthedocs.io/en/latest/


- Installation 

    !pip install datatable

____
### __MODIN__:

- Documentation 

    https://modin.readthedocs.io/en/stable/

- Installation (with dask)

    !pip install modin

!   pip install "dask[distributed]" --upgrade

- Set up (with dask)

    import os

    os.environ['MODIN_ENGINE'] = 'dask'

    import modin.pandas as md
____

### __VAEX__:

- Documentation 

    https://pypi.org/project/vaex/

    It appears that at this point VAEX can not be installed in 3.12.2 on MAC

    https://github.com/vaexio/vaex/issues/2397

- Installation

    !pip install vaex

    or 

    !conda install -c conda-forge vaex

_____
________


In [1]:
#Importing libraries 
import polars as pl
import datatable  as dt
import os
os.environ['MODIN_ENGINE'] = 'dask'
import modin.pandas as md
import modin
print("NUM Partitions available: ", modin.config.NPartitions.get())
import pandas as pd
import time
import numpy as np
from statistics import mean, stdev

NUM Partitions available:  16


In [26]:
def exec_time(code_str):
    start = time.time()
    exec(code_str)
    end = time.time()
    return np.round((end-start)/60*1000, 6)

def functiontiming(cmd_d, metr_nm, dictionary, loop = 10, add_cmd = None):
    print(metr_nm)
    for key, cmd in cmd_d.items():
        counter = 0
        times = []
        try: 
            while counter <= loop:
                if add_cmd:
                    new_cmd = add_cmd[key]
                    exec_time(new_cmd)
                time = exec_time(cmd)
                times.append(time)
                counter +=1
            meant = np.round(mean(times), 6)
            stdevt = np.round(stdev(times), 6)
            maxt = np.round(max(times), 6)
            mint = np.round(min(times), 6)
            add_dic = {'METRIC': metr_nm, 'LIBRARY': key, 'TIME (avg)': meant, "TIME (stdv)": stdevt, "TIME (max)": maxt, "TIME (min)": mint,   'N': loop}

        except Exception as E:
            print(key, " ERROR:\n", str(E))
            add_dic = {}
        add_dic = {'METRIC': metr_nm, 'LIBRARY': key, 'TIME (avg)': meant, "TIME (stdv)": stdevt, "TIME (max)": maxt, "TIME (min)": mint,   'N': loop}
        if metr_nm in dictionary:
            dictionary[metr_nm].append(add_dic)
        else: 
            dictionary[metr_nm] = [add_dic]
        print(key, '\t', meant, 'usec')     

def dict_to_df(dictionary, file_size = None):
    df = pd.DataFrame()
    for k, v in dictionary.items(): 
        df1 = pd.DataFrame(v)
        df = pd.concat([df, df1], axis = 0).reset_index(drop = True)
    if file_size: 
        df.loc[:,'FILE_SZ'] = file_size
    return df  

______ 
### ~SMALL FILE 

In [3]:
# Number of data points to calculate statictics 
loops = 5
file = 'data/data_small.csv'
print("File size: ", np.round(os.stat(file).st_size / (1024 * 1024), 2), "MB")
RESULT_SM = {}

File size:  13.84 MB


In [4]:
cmdsrd = {
        'datatable': 'global dtdf ; dtdf = dt.fread(file)',
        'pandas'   : 'global pdf  ; pdf = pd.read_csv(file, low_memory = False )',
        'polars'   : 'global pldf ; pldf = pl.read_csv(file, infer_schema_length=100000, ignore_errors = True )',
        'modin'    : 'global mdf  ; mdf = md.read_csv(file, low_memory = False)'
        }
metric = 'READ_CSV'
functiontiming(cmdsrd, metric, RESULT_SM, loop = loops)

READ_CSV
datatable 	 2.000678 usec
pandas 	 8.39009 usec
polars 	 7.793179 usec


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


modin 	 30.305312 usec


In [5]:
cmdshp = {
        'datatable': 'dtdf.shape',
        'pandas'   : 'pdf.shape',
        'polars'   : 'pldf.shape',
        'modin'    : 'mdf.shape'
        }
metric = 'PRINT DF SHAPE'

functiontiming(cmdshp, metric, RESULT_SM, loop = loops)

PRINT DF SHAPE
datatable 	 0.000595 usec
pandas 	 0.000617 usec
polars 	 0.000651 usec
modin 	 0.033369 usec


In [6]:
cmds_copy = {
        'datatable': 'global dtdf1 ; dtdf1 = dtdf.copy()',
         'pandas'  : 'global pdf1  ; pdf1 = pdf.copy()',
         'polars'  : 'global pldf1 ; pldf1 = pldf.clone()',
         'modin'   : 'global mdf1  ; mdf1 = mdf.copy()'
        }
metric = 'CREATE COPY'
functiontiming(cmds_copy, metric, RESULT_SM, loop = loops)

CREATE COPY
datatable 	 0.001111 usec
pandas 	 0.305202 usec
polars 	 0.001213 usec
modin 	 0.01527 usec


In [7]:
cmds_col1 = {
         'datatable': 'global dtdf1 ; dtdf1.names = {"CRASH_CRN":"CRASH_CRNnew"}',
         'pandas'   : 'global pdf1  ; pdf1 = pdf1.rename(columns = {"CRASH_CRN":"CRASH_CRNnew"})',
         'polars'   : 'global pldf1 ; pldf1 = pldf1.rename({"CRASH_CRN":"CRASH_CRNnew"})',
         'modin'    : 'global mdf1  ; mdf1 = mdf1.rename(columns = {"CRASH_CRN":"CRASH_CRNnew"})'
        }
metric = 'RENAME SINGLE COLUMN'
functiontiming(cmds_col1, metric, RESULT_SM, add_cmd = cmds_copy, loop = loops)

RENAME SINGLE COLUMN
datatable 	 0.002047 usec
pandas 	 0.20714 usec
polars 	 0.006823 usec
modin 	 0.184049 usec


In [8]:
new_columns = [col+'NEW' for col in pdf.columns]
new_colums_dict = {}
for col in pdf.columns:
    new_colums_dict[col] = col+'NEW'

cmds_col_all = {
         'datatable': 'global dtdf  ; dtdf.names = new_columns',
         'pandas'   : 'global pdf   ; pdf.columns = new_columns',
         # For polars to work with no errors I had to create a new dataframe. 
         # Tests without new copy in other platforms worked with no issues
         'polars'   : 'global pldf2 ; pldf2 =  pldf.rename(new_colums_dict)',
         'modin'    : 'global mdf   ; mdf = mdf.rename(columns = new_colums_dict)'
        }
metric = 'RENAME ALL COLUMNS'
functiontiming(cmds_col_all, metric, RESULT_SM, add_cmd = cmds_copy, loop = loops)

RENAME ALL COLUMNS
datatable 	 0.001423 usec
pandas 	 0.004082 usec
polars 	 0.022623 usec
modin 	 0.220529 usec


In [9]:
cmds_sort1 = {
         'datatable': 'dtdf[:,:, dt.sort("MUNICIPALITYNEW", reverse=True)]',
         'pandas'   : 'pdf.sort_values(by = ["MUNICIPALITYNEW"], ascending = [False])',
         'polars'   : 'pldf2.sort("MUNICIPALITYNEW", descending=True)',
         'modin'    : 'mdf.sort_values(by = ["MUNICIPALITYNEW"], ascending = [False])'
        }
metric = 'SORT ONE COLUMN'
functiontiming(cmds_sort1, metric, RESULT_SM, loop = loops)

SORT ONE COLUMN
datatable 	 0.015366 usec
pandas 	 0.330129 usec
polars 	 0.295716 usec
modin 	 16.854792 usec


In [10]:
cmds_sort2 = {
         'datatable': 'dtdf[:,:, dt.sort(["MUNICIPALITYNEW", "CRASH_YEARNEW"], reverse=[True, False])]',
         'pandas'   : 'pdf.sort_values(by = ["MUNICIPALITYNEW", "CRASH_YEARNEW"], ascending = [False, True])',
         'polars'   : 'pldf2.sort("MUNICIPALITYNEW", "CRASH_YEARNEW", descending=[True, False])',
         'modin'    : 'mdf.sort_values(by = ["MUNICIPALITYNEW", "CRASH_YEARNEW"], ascending = [False, True])'
        }
metric = 'SORT TWO COLUMN'
functiontiming(cmds_sort2, metric, RESULT_SM, loop = loops)

SORT TWO COLUMN
datatable 	 0.05833 usec
pandas 	 0.627631 usec
polars 	 0.36924 usec
modin 	 21.510556 usec


In [11]:
from datatable import dt, f, by
grp_by_sum = {
         'datatable': 'dtdf[:, dt.sum(f.CRASH_YEARNEW), by("MUNICIPALITYNEW")]',
         'pandas'   : 'pdf.groupby("MUNICIPALITYNEW")["CRASH_YEARNEW"].sum()',
         'polars'   : 'pldf2.group_by("MUNICIPALITYNEW").agg(pl.sum("CRASH_YEARNEW"))',
         'modin'    : 'mdf.groupby("MUNICIPALITYNEW")["CRASH_YEARNEW"].sum()'
        }
metric = 'GROUP BY SUM'
functiontiming(grp_by_sum, metric, RESULT_SM, loop = loops)

GROUP BY SUM
datatable 	 0.012406 usec
pandas 	 0.103672 usec
polars 	 0.194096 usec
modin 	 16.158846 usec


In [24]:
dict_to_df(RESULT_SM, 'Small')

,METRIC,LIBRARY,TIME (avg),TIME (stdv),TIME (max),TIME (min),N,FILE_SZ
0,READ_CSV,datatable,2.000678,0.048007,2.093669,1.954083,5,Small
1,READ_CSV,pandas,8.390090,0.770463,9.674168,7.589869,5,Small
2,READ_CSV,polars,7.793179,0.665051,8.737679,7.200384,5,Small
3,READ_CSV,modin,30.305312,31.993676,95.422800,15.128465,5,Small
4,PRINT DF SHAPE,datatable,0.000595,0.000199,0.000966,0.000437,5,Small
5,PRINT DF SHAPE,pandas,0.000617,0.000234,0.001081,0.000465,5,Small
6,PRINT DF SHAPE,polars,0.000651,0.000388,0.001434,0.000453,5,Small
7,PRINT DF SHAPE,modin,0.033369,0.078303,0.193199,0.000830,5,Small
8,CREATE COPY,datatable,0.001111,0.000716,0.002551,0.000715,5,Small
9,CREATE COPY,pandas,0.305202,0.162573,0.524366,0.196421,5,Small


_____

### ~Medium file

In [13]:
### Deleting dataframes used with the Small file data
del dtdf, dtdf1, pdf, pdf1, pldf, pldf1, pldf2, mdf, mdf1

In [14]:
file = 'data/data_medium.csv'
print("File size: ", np.round(os.stat(file).st_size / (1024 * 1024), 2), "MB")
RESULT_MD = {}

File size:  140.68 MB


In [15]:
metric = 'READ_CSV'
functiontiming(cmdsrd, metric, RESULT_MD, loop = loops)
metric = 'PRINT DF SHAPE'
functiontiming(cmdshp, metric, RESULT_MD, loop = loops)
metric = 'CREATE COPY'
functiontiming(cmds_copy, metric, RESULT_MD, loop = loops)
metric = 'RENAME SINGLE COLUMN'
functiontiming(cmds_col1, metric, RESULT_MD, add_cmd = cmds_copy, loop = loops)
metric = 'RENAME ALL COLUMNS'
functiontiming(cmds_col_all, metric, RESULT_MD, add_cmd = cmds_copy, loop = loops)
metric = 'SORT ONE COLUMN'
functiontiming(cmds_sort1, metric, RESULT_MD, loop = loops)
metric = 'SORT TWO COLUMN'
functiontiming(cmds_sort2, metric, RESULT_MD, loop = loops)
metric = 'GROUP BY SUM'
functiontiming(grp_by_sum, metric, RESULT_MD, loop = loops)

READ_CSV
datatable 	 7.599529 usec
pandas 	 128.133898 usec
polars 	 36.982534 usec
modin 	 40.691274 usec
PRINT DF SHAPE
datatable 	 0.000539 usec
pandas 	 0.000534 usec
polars 	 0.000559 usec
modin 	 0.152453 usec
CREATE COPY
datatable 	 0.000871 usec
pandas 	 4.358777 usec
polars 	 0.000925 usec
modin 	 0.0172 usec
RENAME SINGLE COLUMN
datatable 	 0.001506 usec
pandas 	 3.773111 usec
polars 	 0.003044 usec
modin 	 0.166738 usec
RENAME ALL COLUMNS
datatable 	 0.001421 usec
pandas 	 0.003894 usec
polars 	 0.01696 usec
modin 	 0.190247 usec
SORT ONE COLUMN
datatable 	 0.034567 usec
pandas 	 5.794752 usec
polars 	 3.280683 usec
modin 	 25.664983 usec
SORT TWO COLUMN
datatable 	 0.198223 usec
pandas 	 8.182821 usec
polars 	 3.110372 usec
modin 	 18.299376 usec
GROUP BY SUM
datatable 	 0.063098 usec
pandas 	 0.14553 usec
polars 	 1.012639 usec
modin 	 15.972951 usec


In [25]:
dict_to_df(RESULT_MD, 'Medium')

,METRIC,LIBRARY,TIME (avg),TIME (stdv),TIME (max),TIME (min),N,FILE_SZ
0,READ_CSV,datatable,7.599529,0.503172,8.287736,7.077634,5,Medium
1,READ_CSV,pandas,128.133898,4.212348,135.651199,124.041998,5,Medium
2,READ_CSV,polars,36.982534,2.065923,40.177031,35.348948,5,Medium
3,READ_CSV,modin,40.691274,1.281561,42.733868,39.657267,5,Medium
4,READ_CSV,datatable,13.190176,1.366436,15.757235,11.669183,5,Medium
5,READ_CSV,pandas,257.976118,12.288647,278.203801,241.469316,5,Medium
6,READ_CSV,polars,41.723188,2.128056,44.707286,39.940206,5,Medium
7,READ_CSV,modin,78.436092,9.431235,97.543752,73.557818,5,Medium
8,PRINT DF SHAPE,datatable,0.000539,0.000156,0.000830,0.000429,5,Medium
9,PRINT DF SHAPE,pandas,0.000534,0.000117,0.000767,0.000465,5,Medium


_____

### ~Large file

In [17]:
### Deleting dataframes used with the Medium file data
del dtdf, dtdf1, pdf, pdf1, pldf, pldf1, pldf2, mdf, mdf1

In [18]:
file = 'data/data_large.csv'
print("File size: ", np.round(os.stat(file).st_size / (1024 * 1024), 2), "MB")
RESULT_LG = {}

File size:  281.36 MB


In [19]:
metric = 'READ_CSV'
functiontiming(cmdsrd, metric, RESULT_LG, loop = loops)
metric = 'PRINT DF SHAPE'
functiontiming(cmdshp, metric, RESULT_LG, loop = loops)
metric = 'CREATE COPY'
functiontiming(cmds_copy, metric, RESULT_LG, loop = loops)
metric = 'RENAME SINGLE COLUMN'
functiontiming(cmds_col1, metric, RESULT_LG, add_cmd = cmds_copy, loop = loops)
metric = 'RENAME ALL COLUMNS'
functiontiming(cmds_col_all, metric, RESULT_LG, add_cmd = cmds_copy, loop = loops)
metric = 'SORT ONE COLUMN'
functiontiming(cmds_sort1, metric, RESULT_LG, loop = loops)
metric = 'SORT TWO COLUMN'
functiontiming(cmds_sort2, metric, RESULT_LG, loop = loops)
metric = 'GROUP BY SUM'
functiontiming(grp_by_sum, metric, RESULT_LG, loop = loops)

READ_CSV
datatable 	 13.190176 usec
pandas 	 257.976118 usec
polars 	 41.723188 usec
modin 	 78.436092 usec
PRINT DF SHAPE
datatable 	 0.02132 usec
pandas 	 0.001069 usec
polars 	 0.00104 usec
modin 	 0.143733 usec
CREATE COPY
datatable 	 0.001252 usec
pandas 	 11.407617 usec
polars 	 0.001051 usec
modin 	 0.037736 usec
RENAME SINGLE COLUMN
datatable 	 0.001594 usec
pandas 	 10.1069 usec
polars 	 0.016214 usec
modin 	 0.192159 usec
RENAME ALL COLUMNS
datatable 	 0.00145 usec
pandas 	 0.003903 usec
polars 	 0.042649 usec
modin 	 0.248025 usec
SORT ONE COLUMN
datatable 	 0.048209 usec
pandas 	 14.148394 usec
polars 	 9.442515 usec
modin 	 40.68102 usec
SORT TWO COLUMN
datatable 	 0.300782 usec
pandas 	 18.230333 usec
polars 	 7.592119 usec
modin 	 23.675627 usec
GROUP BY SUM
datatable 	 0.132331 usec
pandas 	 0.230779 usec
polars 	 3.108768 usec
modin 	 13.533794 usec


In [28]:
dict_to_df(RESULT_LG, 'Large')

,METRIC,LIBRARY,TIME (avg),TIME (stdv),TIME (max),TIME (min),N,FILE_SZ
0,PRINT DF SHAPE,datatable,0.021320,0.050678,0.124764,0.000429,5,Large
1,PRINT DF SHAPE,pandas,0.001069,0.000262,0.001486,0.000834,5,Large
2,PRINT DF SHAPE,polars,0.001040,0.001150,0.003366,0.000449,5,Large
3,PRINT DF SHAPE,modin,0.143733,0.349819,0.857798,0.000751,5,Large
4,CREATE COPY,datatable,0.001252,0.001086,0.003461,0.000715,5,Large
5,CREATE COPY,pandas,11.407617,4.544324,20.667219,9.330483,5,Large
6,CREATE COPY,polars,0.001051,0.000705,0.002472,0.000664,5,Large
7,CREATE COPY,modin,0.037736,0.059067,0.158270,0.012632,5,Large
8,RENAME SINGLE COLUMN,datatable,0.001594,0.000456,0.002515,0.001347,5,Large
9,RENAME SINGLE COLUMN,pandas,10.106900,0.074261,10.190566,9.995166,5,Large


In [41]:
dict_to_df(RESULT_LG).drop(['N'], axis = 1).merge(dict_to_df(RESULT_MD), on = ['METRIC', 'LIBRARY'], suffixes=['_LG', '_MD']).drop(['N'], axis = 1).merge(dict_to_df(RESULT_SM), on = ['METRIC', 'LIBRARY'])

,METRIC,LIBRARY,TIME (avg)_LG,TIME (stdv)_LG,TIME (max)_LG,TIME (min)_LG,TIME (avg)_MD,TIME (stdv)_MD,TIME (max)_MD,TIME (min)_MD,TIME (avg),TIME (stdv),TIME (max),TIME (min),N
0,PRINT DF SHAPE,datatable,0.021320,0.050678,0.124764,0.000429,0.000539,0.000156,0.000830,0.000429,0.000595,0.000199,0.000966,0.000437,5
1,PRINT DF SHAPE,pandas,0.001069,0.000262,0.001486,0.000834,0.000534,0.000117,0.000767,0.000465,0.000617,0.000234,0.001081,0.000465,5
2,PRINT DF SHAPE,polars,0.001040,0.001150,0.003366,0.000449,0.000559,0.000131,0.000803,0.000453,0.000651,0.000388,0.001434,0.000453,5
3,PRINT DF SHAPE,modin,0.143733,0.349819,0.857798,0.000751,0.152453,0.371236,0.910234,0.000731,0.033369,0.078303,0.193199,0.000830,5
4,CREATE COPY,datatable,0.001252,0.001086,0.003461,0.000715,0.000871,0.000169,0.001188,0.000747,0.001111,0.000716,0.002551,0.000715,5
5,CREATE COPY,pandas,11.407617,4.544324,20.667219,9.330483,4.358777,1.166702,6.552132,3.665431,0.305202,0.162573,0.524366,0.196421,5
6,CREATE COPY,polars,0.001051,0.000705,0.002472,0.000664,0.000925,0.000380,0.001665,0.000679,0.001213,0.000862,0.002921,0.000699,5
7,CREATE COPY,modin,0.037736,0.059067,0.158270,0.012632,0.017200,0.004118,0.022165,0.012501,0.015270,0.002724,0.019316,0.012732,5
8,RENAME SINGLE COLUMN,datatable,0.001594,0.000456,0.002515,0.001347,0.001506,0.000177,0.001852,0.001371,0.002047,0.001199,0.004466,0.001347,5
9,RENAME SINGLE COLUMN,pandas,10.106900,0.074261,10.190566,9.995166,3.773111,0.046366,3.848183,3.739552,0.207140,0.020835,0.246433,0.190337,5
